`Django`는 파이썬 웹프레임워크로서 권장 디렉토리 구조가 있습니다. 그런데, 간단한 장고 코드 검증을 위해 `Jupyter Notebook` 밖에서 장고 프로젝트를 생성/세팅하는 것은 지금 제게 너무 번거로운 일입니다. **저는 Jupyter Notebook 밖으로 나가고 싶지 않습니다.** 노트북 안에서 모든 것을 다 끝내고 싶습니다. ㅎㅎ :D

**이 모든 것이 가능합니다.**

본 문서의 코드는 다음 환경에서 테스트되었습니다.

+ Python 3.X
+ Django 1.11.X

Tip: 실제 장고 프로젝트에서 프로젝트와 연동되는 Jupyter Notebook을 쓰고자 하신다면 `django-extensions`의 `shell_plus` 명령을 써보세요. ([공식문서](http://django-extensions.readthedocs.io/en/latest/shell_plus.html))

## 개발 환경 세팅

자. 시작해봅시다.

노트북 서버를 실행하기 전에, 장고를 먼저 설치해주세요.

```
쉘> pip3 install django<=1.11.6
```

그리고, 노트북 서버를 실행시켜주세요. jupyter notebook은 이미 설치되어있으시겠죠? :D

```
쉘> jupyter notebook
```

새로운 Python3 노트북을 만들어주시고, 아래 코드를 차근차근 실행시켜보세요.

## 최소한의 settings

본 노트북에서 필요한 최소한의 장고 `settings` 내역을 세팅해줍니다.

Model을 쓸려면 데이터베이스가 필요한데요. 이 역시 별도 DB세팅을 하는 것은 **사치**이므로, SQLite3를 메모리 베이스로 세팅했습니다.

In [1]:
import django
import os

SECRET_KEY = 'askdjango'    # 임의 문자열
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': ':memory:',
    }
}
ROOT_URLCONF = '__main__'

urls = []

os.environ['DJANGO_SETTINGS_MODULE'] = '__main__'

django.setup()

제 장고 버전은요 ...

In [2]:
django.get_version()

'1.11.4'

## Form 클래스를 정의하고 써봅시다.

In [5]:
from django import forms

def odd_validator(value):
    if value % 2 == 0:
        raise forms.ValidationError('내가 짝수라니 !!!')

class QuizForm(forms.Form):
    answer = forms.IntegerField(validators=[odd_validator])

In [6]:
data = {'answer': 10}

In [7]:
form = QuizForm(data)

`answer`값이 짝수라서, 유효성 검사 실패 !!!

In [8]:
form.is_valid()

False

이렇게 에러 내용 확인도 차암~ 쉽죠?

In [9]:
form.errors

{'answer': ['내가 짝수라니 !!!']}

## Modle 클래스를 정의하고 써봅시다.

In [10]:
from django.db import models

class Post(models.Model):
    title = models.CharField(max_length=100)
    content = models.TextField()
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    class Meta:
        app_label = 'api'  # 앱이 따로 없으므로, app label을 필히 지정해줘야합니다.
    
    def __str__(self):
        return self.title

Jupyter Notebook 내에서만 장고 프로젝트를 운영하다보니 마이그레이션을 할 여건이 안 되어서, 모델 클래스 내역대로 Raw SQL로 DB 테이블 생성해보겠습니다.

In [11]:
from django.db import connection

table_name = Post._meta.db_table

with connection.cursor() as cursor:
    cursor.execute('''
CREATE TABLE "{}"
    ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT,
     "title" varchar(100) NOT NULL,
     "content" text NOT NULL,
     "created_at" datetime NOT NULL,
     "updated_at" datetime NOT NULL);
'''.format(table_name))

생성된 테이블을 확인해보죠. ㅎㅎ 

In [12]:
with connection.cursor() as cursor:
    cursor.execute("select name from sqlite_master where type = 'table';")
    for row in cursor.fetchall():
        print(row)

('api_post',)
('sqlite_sequence',)


이제 Post 모델을 통해 DB에 데이터를 추가해봅시다.

In [13]:
Post.objects.create(
    title='횡단보도 보행자 없으면 우회전 가능?…혼란 빚은 까닭',
    content='교차로에서 우회전할 때 횡단 보도를 건너는 사람이 없다면 보행자 신호가 녹색이더라도 진입할 수 있을까요? 이 문제를 놓고 대법원과 경찰의 판단이 다른 상황입니다.')

<Post: 횡단보도 보행자 없으면 우회전 가능?…혼란 빚은 까닭>

In [14]:
Post.objects.create(
    title="'디지털세대, 아날로그에 빠지다'…아날로그 인기 이유는?",
    content='옛 방식을 고집하는 아날로그 공간들이 젊은 층을 중심으로 주목받고 있습니다.')

<Post: '디지털세대, 아날로그에 빠지다'…아날로그 인기 이유는?>

In [15]:
Post.objects.create(
    title='저녁 줄였는데 누구는 살 빠지고, 난 안 빠지고…이유는',
    content='늦은 시간에 야식 먹으면 다 살로 간다고 하죠? 그래서 야식 증후군이란 말까지 생겼습니다. 또 아침은 많이 먹고 저녁은 되도록 적게 먹는 것이 다이어트의 지름길이라고 생각하기도 합니다. 이게 다 얼마나 맞는 말일까요?')

<Post: 저녁 줄였는데 누구는 살 빠지고, 난 안 빠지고…이유는>

데이터가 잘 들어갔나요? ㅎㅎ

In [16]:
Post.objects.all()

<QuerySet [<Post: 횡단보도 보행자 없으면 우회전 가능?…혼란 빚은 까닭>, <Post: '디지털세대, 아날로그에 빠지다'…아날로그 인기 이유는?>, <Post: 저녁 줄였는데 누구는 살 빠지고, 난 안 빠지고…이유는>]>

## ModelForm이 빠질 수 없죠.

In [17]:
class PostForm(forms.ModelForm):
    class Meta:
        model = Post
        fields = '__all__'

In [18]:
form = PostForm({'title': 'hello'})

In [19]:
form.is_valid()

False

In [20]:
form.errors

{'content': ['This field is required.']}

---

이렇게 **Jupyter Notebook** 내에서의 장고 세팅에 대해서 살펴봤습니다. 장고 개념이 약하실 때, 이런 저런 코드를 테스트해보기 좋은 듯 하네요.

감사합니다.

\- AskDjango 이진석 드림